In [42]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [43]:
from smooth.adam_general.core.adam import Adam

import numpy as np
import pandas as pd

In [44]:
from typing import List, Union, Dict, Any
from smooth.adam_general._adam_general import adam_fitter, adam_forecaster
from numpy.linalg import eigvals
import nlopt

In [45]:
#### Global level and ETS(A,N,N) ####
np.random.seed(33)  # For reproducibility
n_points = 100
time_series = np.random.normal(100, 10, n_points)  # Random errors
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [46]:
adam = Adam(model="ANN", lags=[12])
adam.fit(ts_df, h = 12)
adam.predict()

,mean,lower_0.025,upper_0.975
2031-05-31,99.619575,NaN,NaN
2031-06-30,99.619575,NaN,NaN
2031-07-31,99.619575,NaN,NaN
2031-08-31,99.619575,NaN,NaN
2031-09-30,99.619575,NaN,NaN
2031-10-31,99.619575,NaN,NaN
2031-11-30,99.619575,NaN,NaN
2031-12-31,99.619575,NaN,NaN
2032-01-31,99.619575,NaN,NaN
2032-02-29,99.619575,NaN,NaN


In [47]:
#### Local level and ETS(A,N,N) ####
np.random.seed(41)  # For reproducibility
n_points = 120
errors = np.random.normal(0, 10, n_points)  # Random errors
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    # This is SES with alpha=0.1, just in an alternative form
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + errors[i+1]
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))

In [48]:
# ETS(A,N,N) with alpha=0.1
model = Adam(model="ANN", lags=[12], persistence=0.1)
model.fit(ts_df, h=12)
print("ETS(A,N,N) with alpha=0.1:")
print(model.predict())

# ETS(A,N,N) with estimated alpha 
model = Adam(model="ANN", lags=[12])
model.fit(ts_df, h=12)
print("\nETS(A,N,N) with estimated parameters:")
print(model.adam_estimated["B"])
print(model.predict())

# ETS(M,N,N) with estimated alpha
model = Adam(model="MNN", lags=[12], distribution="dnorm")
model.fit(ts_df, h=12)
print("\nETS(M,N,N) with estimated parameters:")
print(model.predict())


ETS(A,N,N) with alpha=0.1:
                 mean  lower_0.025  upper_0.975
2033-01-31  83.225986          NaN          NaN
2033-02-28  83.225986          NaN          NaN
2033-03-31  83.225986          NaN          NaN
2033-04-30  83.225986          NaN          NaN
2033-05-31  83.225986          NaN          NaN
2033-06-30  83.225986          NaN          NaN
2033-07-31  83.225986          NaN          NaN
2033-08-31  83.225986          NaN          NaN
2033-09-30  83.225986          NaN          NaN
2033-10-31  83.225986          NaN          NaN
2033-11-30  83.225986          NaN          NaN
2033-12-31  83.225986          NaN          NaN

ETS(A,N,N) with estimated parameters:
[6.53178667e-02 9.69906981e+01]
                 mean  lower_0.025  upper_0.975
2033-01-31  84.708881          NaN          NaN
2033-02-28  84.708881          NaN          NaN
2033-03-31  84.708881          NaN          NaN
2033-04-30  84.708881          NaN          NaN
2033-05-31  84.708881          NaN    

In [49]:
#### Local trend and ETS(A,A,N) ####
np.random.seed(42)  # For reproducibility
n_points = 120
errors = np.random.normal(0, 10, n_points)  # Random errors
trend = np.random.normal(0.5, 2, n_points)  # Random trend
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    # This is SES with alpha=0.1, just in an alternative form
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + trend[i] + errors[i+1]
ts_df = pd.DataFrame({'value': time_series}, index=pd.date_range(start='2023-01-01', periods=n_points, freq='ME'))


In [50]:
# ETS(A,A,N) with fixed parameters
model = Adam(model="AAN", lags=[12], persistence=[0.1, 0.02])
model.fit(ts_df, h=12)
print("ETS(A,A,N):")
print(model.predict())

# ETS(A,Ad,N) with estimated parameters
model = Adam(model="AAdN", lags=[12])
model.fit(ts_df, h=12)
print("\nETS(A,Ad,N) estimated parameters:")
print(model.adam_estimated["B"])

# ETS(A,Ad,N) with fixed parameters
model = Adam(model="AAdN", lags=[12], persistence=[0.1, 0.02])
model.fit(ts_df, h=12)
print("\nETS(A,Ad,N):")
print(model.predict())

ETS(A,A,N):
                  mean  lower_0.025  upper_0.975
2033-01-31  165.255525          NaN          NaN
2033-02-28  165.763955          NaN          NaN
2033-03-31  166.272384          NaN          NaN
2033-04-30  166.780814          NaN          NaN
2033-05-31  167.289244          NaN          NaN
2033-06-30  167.797674          NaN          NaN
2033-07-31  168.306104          NaN          NaN
2033-08-31  168.814534          NaN          NaN
2033-09-30  169.322963          NaN          NaN
2033-10-31  169.831393          NaN          NaN
2033-11-30  170.339823          NaN          NaN
2033-12-31  170.848253          NaN          NaN

ETS(A,Ad,N) estimated parameters:
[ 1.68275229e-01  1.04248642e-02  9.86301308e-01  1.04774858e+02
 -3.80921118e-01]

ETS(A,Ad,N):
                  mean  lower_0.025  upper_0.975
2033-01-31  163.235667          NaN          NaN
2033-02-28  163.701313          NaN          NaN
2033-03-31  164.140767          NaN          NaN
2033-04-30  164.555501 

In [51]:
# ETS(M,A,N) with fixed parameters
model = Adam(model="MAN", lags=[12], persistence=[0.1, 0.02])
model.fit(ts_df, h=12)
print("ETS(M,A,N):")
print(model.predict())

# ETS(M,Ad,N) with estimated parameters
model = Adam(model="MAdN", lags=[12])
model.fit(ts_df, h=12)
print("\nETS(M,Ad,N) estimated parameters:")
print(model.adam_estimated["B"])

# ETS(M,Ad,N) with fixed parameters
model = Adam(model="MAdN", lags=[12], persistence=[0.1, 0.02])
model.fit(ts_df, h=12)
print("\nETS(M,Ad,N):")
print(model.predict())

ETS(M,A,N):
                  mean  lower_0.025  upper_0.975
2033-01-31  165.256290          NaN          NaN
2033-02-28  165.764739          NaN          NaN
2033-03-31  166.273187          NaN          NaN
2033-04-30  166.781636          NaN          NaN
2033-05-31  167.290085          NaN          NaN
2033-06-30  167.798533          NaN          NaN
2033-07-31  168.306982          NaN          NaN
2033-08-31  168.815430          NaN          NaN
2033-09-30  169.323879          NaN          NaN
2033-10-31  169.832328          NaN          NaN
2033-11-30  170.340776          NaN          NaN
2033-12-31  170.849225          NaN          NaN

ETS(M,Ad,N) estimated parameters:
[ 1.53638857e-01  1.33971319e-02  9.73336479e-01  1.06238427e+02
 -5.50792615e-01]

ETS(M,Ad,N):
                  mean  lower_0.025  upper_0.975
2033-01-31  163.344657          NaN          NaN
2033-02-28  163.813627          NaN          NaN
2033-03-31  164.258441          NaN          NaN
2033-04-30  164.680342 

In [52]:
# ETS(M,M,N) with fixed parameters
model = Adam(model="MMN", lags=[12], persistence=[0.1, 0.02])
model.fit(ts_df, h=12)
print("ETS(M,M,N):")
print(model.predict())

# ETS(M,Md,N) with estimated parameters
model = Adam(model="MMdN", lags=[12])
model.fit(ts_df, h=12)
print("\nETS(M,Md,N) estimated parameters:")
print(model.adam_estimated)

# ETS(M,Md,N) with fixed parameters
model = Adam(model="MMdN", lags=[12], persistence=[0.1, 0.02])
model.fit(ts_df, h=12)
print("\nETS(M,Md,N):")
print(model.predict())


CFValue is NaN
CFValue is NaN
CFValue is NaN
CFValue is NaN
ETS(M,M,N):


/home/isvetunkov/Cursor/smooth/python/.venv/lib/python3.12/site-packages/numpy/_core/fromnumeric.py:3860: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/isvetunkov/Cursor/smooth/python/.venv/lib/python3.12/site-packages/numpy/_core/_methods.py:145: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/isvetunkov/Cursor/smooth/python/smooth/adam_general/core/forecaster.py:29: UserWarning: Something went wrong in the estimation of the model and NaNs were produced. If this is a mixed model, consider using the pure ones instead.
  warnings.warn("Something went wrong in the estimation of the model and NaNs were produced. "


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices